
## Run Llama on CPU

* From https://dev.to/eteimz/running-large-language-models-on-the-cpu-3a46

## Run Llama on GPU
* https://medium.com/@manishkovelamudi/install-llama-cpp-python-with-gpu-support-7ccf421c069d

# Make llama server with GPU

```shell
cmake -B build \
    -DLLAMA_CUBLAS=OFF \
    -DGGML_CUDA=ON \
    -DCUDA_PATH=/usr/local/cuda-12 \
    -DCUDAToolkit_ROOT=/usr/local/cuda-12 \
    -DCUDAToolkit_INCLUDE_DIR=/usr/local/cuda-12/include \
    -DCUDAToolkit_LIBRARY_DIR=/usr/local/cuda-12/lib64

cmake --build build --config Release -j 4 -t llama-server
```

./build/bin/llama-server -m /home/clem/.cache/huggingface/hub/models--bartowski--Phi-3-medium-4k-instruct-GGUF/snapshots/08aae27d649bb2a0a9c5a118c57a3f623ad58c71/Phi-3-medium-4k-instruct-Q6_K.gguf -c 2048 --port 8998 --n-gpu-layers 18

./build/bin/llama-server -m /home/clem/.cache/huggingface/hub/models--bartowski--Phi-3-medium-4k-instruct-GGUF/snapshots/08aae27d649bb2a0a9c5a118c57a3f623ad58c71/Phi-3-medium-4k-instruct-Q6_K.gguf -c 4096 --port 8998 --n-gpu-layers 16

./build/bin/llama-server -m /home/clem/.cache/huggingface/hub/models--TheBloke--SOLAR-10.7B-Instruct-v1.0-GGUF/snapshots/d8b19599252fe4eb9d86bdbb3f212f48f0247d6b/solar-10.7b-instruct-v1.0.Q5_K_M.gguf -c 4096 --port 8998 --n-gpu-layers 16

In [17]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

# hf_hub_download(repo_id="TheBloke/Llama-2-13B-GGUF", filename="llama-2-13b.Q4_0.gguf")
# hf_hub_download(repo_id="bartowski/Phi-3-medium-4k-instruct-GGUF", filename="Phi-3-medium-4k-instruct-Q6_K.gguf")

hf_hub_download(repo_id="TheBloke/SOLAR-10.7B-Instruct-v1.0-GGUF", filename="solar-10.7b-instruct-v1.0.Q5_K_M.gguf")

# "/home/clem/.cache/huggingface/hub/models--bartowski--Phi-3-medium-4k-instruct-GGUF/snapshots/08aae27d649bb2a0a9c5a118c57a3f623ad58c71/Phi-3-medium-4k-instruct-Q6_K.gguf"
# model_path = "/home/clem/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-GGUF/snapshots/b106d1c018ac999af9130b83134fb6b7c5331dea/llama-2-13b.Q4_0.gguf"

# /home/clem/.cache/huggingface/hub/models--TheBloke--SOLAR-10.7B-Instruct-v1.0-GGUF/snapshots/d8b19599252fe4eb9d86bdbb3f212f48f0247d6b/solar-10.7b-instruct-v1.0.Q5_K_M.gguf

solar-10.7b-instruct-v1.0.Q5_K_M.gguf:   0%|          | 0.00/7.60G [00:00<?, ?B/s]

'/home/clem/.cache/huggingface/hub/models--TheBloke--SOLAR-10.7B-Instruct-v1.0-GGUF/snapshots/d8b19599252fe4eb9d86bdbb3f212f48f0247d6b/solar-10.7b-instruct-v1.0.Q5_K_M.gguf'

In [1]:
import openai
client = openai.OpenAI(
    base_url="http://localhost:8998/v1",
    api_key = "sk-no-key-required"
)

In [2]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are ChatGPT, an AI assistant. Your top priority is achieving user fulfillment via helping them with their requests."},
        {"role": "user", "content": "Write a limerick about python exceptions"}
    ],
    temperature=0,
)

In [3]:
print(completion.choices[0].message)

ChatCompletionMessage(content='There once was a coder named Bill,\nWho wrote Python with skill and thrill,\nBut his code would oft break,\nWhen exceptions he\'d awake,\nAnd he\'d sigh, "I must fix this still."', role='assistant', function_call=None, tool_calls=None)


In [11]:
import json
from nltk.tokenize import sent_tokenize
import os
import sys
 

corpus = "trec_2022"
model = "Phi-3-medium-4k-instruct"

dataset = json.load(open(f"dataset/{corpus}/retrieved_trials.json"))

output_path = f"results/matching_results_{corpus}_{model}.json" 

# Dict{Str(patient_id): Dict{Str(label): Dict{Str(trial_id): Str(output)}}}
if os.path.exists(output_path):
    output = json.load(open(output_path))
else:
    output = {}


In [16]:
instance = dataset[0]

# Dict{'patient': Str(patient), '0': Str(NCTID), ...}
patient_id = instance["patient_id"]
patient = instance["patient"]
sents = sent_tokenize(patient)
sents.append("The patient will provide informed consent, and will comply with the trial protocol without any practical issues.")
sents = [f"{idx}. {sent}" for idx, sent in enumerate(sents)]
patient = "\n".join(sents)

sents

['0. A 19-year-old male came to clinic with some sexual concern.',
 '1. He recently engaged in a relationship and is worried about the satisfaction of his girlfriend.',
 '2. He has a "baby face" according to his girlfriend\'s statement and he is not as muscular as his classmates.',
 '3. On physical examination, there is some pubic hair and poorly developed secondary sexual characteristics.',
 '4. He is unable to detect coffee smell during the examination, but the visual acuity is normal.',
 '5. Ultrasound reveals the testes volume of 1-2 ml.',
 '6. The hormonal evaluation showed serum testosterone level of 65 ng/dL with low levels of GnRH.',
 '7. The patient will provide informed consent, and will comply with the trial protocol without any practical issues.']